In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('C:/Users/91740/Documents/SheepFinance/FBIL.xlsx')

In [3]:
df

,Tenor (Year),YTM% p.a.(Semi-Annual),YTM % p.a.(Annualized)
0,0.25,0.063562,0.065674
1,0.50,0.065520,0.067440
2,0.75,0.066541,0.068325
3,1.00,0.068232,0.069396
4,1.25,0.068737,0.069918
...,...,...,...
155,39.00,0.074352,0.075734
156,39.25,0.074343,0.075725
157,39.50,0.074360,0.075742
158,39.75,0.074355,0.075737


In [4]:
def input_file(df):
    zero_rates = df['YTM % p.a.(Annualized)']
    tenors = df['Tenor (Year)']
    return(zero_rates, tenors)

In [5]:
def NPV_3m(r):
    x = df.iloc[0]
    fix_leg = []
    float_leg = []
    tenor_check = x['Tenor (Year)']
    Par_SemiAnnual = x['YTM% p.a.(Semi-Annual)']
    t = 0.25
    
    i = 0
    zero_rates[i] = r
    zr = np.interp(t, tenors, zero_rates)
    disc_factor = 1/(1+zr)**t
    fra = (1/(disc_factor)-1)/0.25
    
    fix_leg = 1e7*Par_SemiAnnual*0.25*disc_factor
    float_leg = 1e7*fra*0.25*disc_factor
    npv = fix_leg - float_leg
    return(npv)

In [6]:
def newton_raphson_3m(r):
    h = 0.0000001
    for i in range(0,10):
        f1 = NPV_3m(r+h)
        f2 = NPV_3m(r)
        df = (f1-f2)/h
        r = r - f2/df
    return(r)    

In [7]:
def NPV_6m(r):
    x = df.iloc[1]
    fix_leg = []
    float_leg = []
    tenor_check = x['Tenor (Year)']
    Par_SemiAnnual = x['YTM% p.a.(Semi-Annual)']
    t = 0.5
    
    i = 1
    zero_rates[i] = r
    zr = np.interp(t, tenors, zero_rates)
    disc_factor = 1/(1+zr)**t
    fra = (1/(disc_factor)-1)/0.5
    
    fix_leg = 1e7*Par_SemiAnnual*0.5*disc_factor
    float_leg = 1e7*fra*0.5*disc_factor
    npv = fix_leg - float_leg
    return(npv)

In [8]:
def newton_raphson_6m(r):
    h = 0.0000001
    for i in range(0,10):
        f1 = NPV_6m(r+h)
        f2 = NPV_6m(r)
        df = (f1-f2)/h
        r = r - f2/df
    return(r)    

In [9]:
def NPV_full(i,r):
    
    zero_rates[i] = r
    
    fix_leg = []
    float_leg = []
    x = df.iloc[i]
    tenor_check = x['Tenor (Year)']
    Par_SemiAnnual = x['YTM% p.a.(Semi-Annual)']
    n = tenor_check % 0.5
    if(n != 0):
        m = int((tenor_check - 0.25)/0.5)
        m = m + 1
        t = 0
        for j in range(0,m):
            if(j==0):
                t = 0.25
                zr1 = np.interp(0.25, tenors, zero_rates)
                zr2 = np.interp(t, tenors, zero_rates)
                disc_factor1 = 1/(1+zr1)**(t-0.25)
                disc_factor2 = 1/(1+zr2)**t
                fra = ((disc_factor1)/(disc_factor2)-1)*1/0.25
                fix_leg.append(1e7*Par_SemiAnnual*0.25*disc_factor2)
                float_leg.append(1e7*fra*0.25*disc_factor2)
                
            else:
                t = 0.5 + t
                zr1 = np.interp(t - 0.5, tenors, zero_rates)
                zr2 = np.interp(t, tenors, zero_rates)
                disc_factor1 = 1/(1+zr1)**(t-0.5)
                disc_factor2 = 1/(1+zr2)**t
                fra = ((disc_factor1)/(disc_factor2)-1)*1/0.5
                fix_leg.append(1e7*Par_SemiAnnual*0.5*disc_factor2)
                float_leg.append(1e7*fra*0.5*disc_factor2)
                
        npv = sum(fix_leg) - sum(float_leg)
        
    if(n == 0):   
        m = int((tenor_check - 0.5)/0.5)
        m = m + 1
        t = 0
        for j in range(0,m):
            t = 0.5 + t
            zr1 = np.interp(t - 0.5, tenors, zero_rates)
            zr2 = np.interp(t, tenors, zero_rates)
            disc_factor1 = 1/(1+zr1)**(t-0.5)
            disc_factor2 = 1/(1+zr2)**t
            fra = ((disc_factor1)/(disc_factor2)-1)*1/0.5
            fix_leg.append(1e7*Par_SemiAnnual*0.5*disc_factor2)
            float_leg.append(1e7*fra*0.5*disc_factor2)

        npv = sum(fix_leg) - sum(float_leg)
        
    return(npv)    
            

In [10]:
def newton_raphson_full(i,r):
    h = 0.0000001
    for j in range(0,8):
        f1 = NPV_full(i,r+h)
        f2 = NPV_full(i,r)
        df = (f1-f2)/h
        r = r - f2/df
    return(r)  

In [11]:
zero_rates, tenors = input_file(df)
r = 0.05
for p in range(0, len(df)):
    if(p == 0): 
        zero_rates[p] = newton_raphson_3m(r)
    elif(p == 1): 
        zero_rates[p] = newton_raphson_6m(r)
    else: 
        zero_rates[p] = newton_raphson_full(p,r) 
    

In [12]:
zero_rates

0      0.065094
1      0.066593
2      0.067856
3      0.069444
4      0.070106
         ...   
155    0.076850
156    0.076861
157    0.076886
158    0.076919
159    0.076918
Name: YTM % p.a.(Annualized), Length: 160, dtype: float64